In [12]:
import pandas as pd
from sklearn.metrics import f1_score, classification_report
import ast
import random
import os
import cohere
import google.generativeai as genai
from tqdm import tqdm
import time
import re

# STEP 1: Load and sample dataset
df = pd.read_csv("Datasets/journal_reddit_posts_multilabel.csv").dropna(subset=['text'])

label_cols = ['label_0', 'label_1', 'label_2', 'label_3']
label_map = ['neutral', 'low mood/depressed', 'anxious/worried', 'stressed/overwhelmed']

# STEP 2: Sample 1000 examples randomly
subset_df = df.sample(n=1000, random_state=42).reset_index(drop=True)

# STEP 4: Convert ground truth labels to string sets for easy comparison
def label_vector_to_list(row):
    return [label_names[i] for i, val in enumerate(row) if val == 1]

subset_df['ground_truth'] = subset_df[label_cols].values.tolist()
subset_df['ground_truth'] = subset_df['ground_truth'].apply(lambda vec: label_vector_to_list(vec))

# STEP 5: Placeholder for model predictions (Gemini/Cohere will go here)
subset_df['gemini_pred'] = None  # Fill this later with model outputs
subset_df['cohere_pred'] = None  # Fill this later with model outputs

In [ ]:
# Set your API keys
COHERE_API_KEY = "YOUR_API_KEY"

# Configure Gemini
genai.configure(api_key="YOUR_API_KEY")
model = genai.GenerativeModel("gemini-2.5-flash")

# Configure Cohere
cohere_client = cohere.ClientV2(COHERE_API_KEY)

# 🧠 Build prompt requesting numbered labels only
def build_numbered_prompt(texts):
    header = (
        "You are a mental health sentiment analyzer.\n"
        "Each text may reflect one or more emotional states.\n"
        "Possible labels:\n"
        "0 = neutral, 1 = low mood/depressed, 2 = anxious/worried, 3 = stressed/overwhelmed\n"
        "For each text, return only a comma-separated list of label numbers.\n"
        "Format:\n1. 1, 2\n2. 0\n3. 1, 3\n\n"
    )
    batched = ""
    for i, text in enumerate(texts, 1):
        batched += f"{i}. Text: {text.strip()}\n"
    return header + batched

# 🧠 Extract numerical predictions like "1. 1, 2"
def extract_numbered_label_preds(raw_output, expected_count):
    numbered_pattern = re.compile(r"^\s*(\d+)[\.\)]\s*([0-3](?:\s*,\s*[0-3])*)$")
    results = []
    for line in raw_output.strip().split("\n"):
        match = numbered_pattern.match(line.strip())
        if match:
            pred_str = match.group(2)
            preds = [int(x.strip()) for x in pred_str.split(",") if x.strip().isdigit()]
            results.append(preds)
    while len(results) < expected_count:
        results.append([])
    return results

# ✅ Gemini batched predictions
def get_gemini_predictions(texts, batch_size=5):
    preds = []
    for i in tqdm(range(0, len(texts), batch_size), desc="Gemini batches"):
        batch = texts[i:i+batch_size]
        prompt = build_numbered_prompt(batch)
        try:
            response = model.generate_content(prompt)
            batch_preds = extract_numbered_label_preds(response.text, len(batch))
        except Exception as e:
            print(f"[Gemini] Error at batch {i}: {e}")
            batch_preds = [[] for _ in batch]
        preds.extend(batch_preds)
        time.sleep(5)
    return preds

# ✅ Cohere batched predictions
def get_cohere_predictions(texts, batch_size=5):
    preds = []
    for i in tqdm(range(0, len(texts), batch_size), desc="Cohere batches"):
        batch = texts[i:i+batch_size]
        prompt = build_numbered_prompt(batch)
        try:
            response = cohere_client.chat(
                model="command-a-03-2025",
                messages=[{"role":"user", "content": prompt}]
            )
            batch_preds = extract_numbered_label_preds(response.message.content[0].text, len(batch))
        except Exception as e:
            print(f"[Cohere] Error at batch {i}: {e}")
            batch_preds = [[] for _ in batch]
        preds.extend(batch_preds)
        time.sleep(8)
    return preds

In [14]:
# Assuming subset_df already created and contains 'prompt' column
gemini_preds = get_gemini_predictions(subset_df['text'], batch_size=5)

subset_df['gemini_pred'] = gemini_preds

Cohere batches:   0%|                                  | 0/200 [00:00<?, ?it/s]

[Cohere] Error at batch 0: BaseCohere.generate() got an unexpected keyword argument 'prompts'


Cohere batches:   0%|▏                         | 1/200 [00:05<16:35,  5.00s/it]

[Cohere] Error at batch 5: BaseCohere.generate() got an unexpected keyword argument 'prompts'


Cohere batches:   1%|▎                         | 2/200 [00:10<16:30,  5.00s/it]

...

Cohere batches:  99%|███████████████████████▊| 198/200 [16:31<00:10,  5.01s/it]

[Cohere] Error at batch 990: BaseCohere.generate() got an unexpected keyword argument 'prompts'


Cohere batches: 100%|███████████████████████▉| 199/200 [16:36<00:05,  5.01s/it]

[Cohere] Error at batch 995: BaseCohere.generate() got an unexpected keyword argument 'prompts'


Cohere batches: 100%|████████████████████████| 200/200 [16:41<00:00,  5.01s/it]


In [14]:
# Assuming subset_df already created and contains 'prompt' column
gemini_preds = get_gemini_predictions(subset_df['text'], batch_size=5)

subset_df['gemini_pred'] = gemini_preds

Cohere batches:   0%|                                  | 0/200 [00:00<?, ?it/s]

[Cohere] Error at batch 0: BaseCohere.generate() got an unexpected keyword argument 'prompts'


Cohere batches:   0%|▏                         | 1/200 [00:05<16:35,  5.00s/it]

[Cohere] Error at batch 5: BaseCohere.generate() got an unexpected keyword argument 'prompts'


Cohere batches:   1%|▎                         | 2/200 [00:10<16:30,  5.00s/it]

...

Cohere batches:  99%|███████████████████████▊| 198/200 [16:31<00:10,  5.01s/it]

[Cohere] Error at batch 990: BaseCohere.generate() got an unexpected keyword argument 'prompts'


Cohere batches: 100%|███████████████████████▉| 199/200 [16:36<00:05,  5.01s/it]

[Cohere] Error at batch 995: BaseCohere.generate() got an unexpected keyword argument 'prompts'


Cohere batches: 100%|████████████████████████| 200/200 [16:41<00:00,  5.01s/it]


In [21]:
cohere_preds = get_cohere_predictions(subset_df['text'], batch_size=5)

subset_df['cohere_pred'] = cohere_preds

Cohere batches: 100%|████████████████████████| 200/200 [31:51<00:00,  9.56s/it]


In [23]:
cohere_preds

[[0],
 [2, 3],
 [2],
 [1, 3],
 [1, 3],
 [0],
 [2],
 [3],
 [2],
 [0],
 [1, 3],
 [0],
 [0],
 [0],
 [0],
 [1, 2, 3],
 [2],
 [1, 2, 3],
 [1, 3],
 [1, 2],
 [1, 3],
 [2],
 [0],
 [0],
 [1],
 [0],
 [1, 2],
 [2, 3],
 [1, 2, 3],
 [1],
 [1, 2],
 [1, 2, 3],
 [1, 2],
 [1, 3],
 [0],
 [1, 3],
 [1, 2],
 [2],
 [0],
 [1, 2],
 [1, 2, 3],
 [2],
 [1],
 [3],
 [0],
 [1],
 [1, 2, 3],
 [2],
 [1, 2, 3],
 [0],
 [0],
 [1],
 [1, 3],
 [2],
 [1, 2, 3],
 [1, 2, 3],
 [1, 2],
 [1, 3],
 [1, 2, 3],
 [0],
 [1, 2],
 [1, 3],
 [1, 2, 3],
 [1],
 [0],
 [1, 2],
 [1, 2],
 [0],
 [1, 3],
 [2, 3],
 [1, 2, 3],
 [1],
 [1],
 [0, 3],
 [1, 3],
 [1],
 [2],
 [1, 3],
 [1, 3],
 [0],
 [1, 3],
 [0],
 [0],
 [2],
 [0],
 [1, 2, 3],
 [0],
 [2, 3],
 [1],
 [1, 2, 3],
 [1, 2],
 [1, 3],
 [1],
 [2],
 [1, 2, 3],
 [1, 2, 3],
 [1],
 [1, 3],
 [1, 3],
 [0],
 [1, 2],
 [1],
 [1, 3],
 [2],
 [1],
 [1, 2, 3],
 [1, 2],
 [1, 2],
 [1, 2],
 [0],
 [1],
 [1],
 [0],
 [1, 2, 3],
 [0],
 [1, 3],
 [1, 2],
 [0],
 [0],
 [1, 2],
 [1, 2, 3],
 [2, 3],
 [2],
 [1, 3],
 [2],
 [1]

In [24]:
from sklearn.metrics import f1_score

# Label index to name (in case needed for interpretability)
label_names = ['neutral', 'low mood', 'anxious', 'stressed']
label_cols = ['label_0', 'label_1', 'label_2', 'label_3']  # assumed ground truth one-hot columns

# Convert list of predicted label indices to binary format
def convert_preds_to_binary(preds):
    binary = []
    for pred in preds:
        row = [0] * len(label_names)
        for label in pred:
            if isinstance(label, int) and 0 <= label < len(label_names):
                row[label] = 1
        binary.append(row)
    return binary

# Evaluate macro/weighted F1
def evaluate_predictions(df, column_name):
    pred_binary = convert_preds_to_binary(df[column_name])
    true_binary = df[label_cols].astype(int).values.tolist()
    macro = f1_score(true_binary, pred_binary, average='macro')
    weighted = f1_score(true_binary, pred_binary, average='weighted')
    return macro, weighted

# Usage
macro_g, weighted_g = evaluate_predictions(subset_df, 'gemini_pred')
macro_c, weighted_c = evaluate_predictions(subset_df, 'cohere_pred')

print(f"Gemini - Macro F1: {macro_g:.4f}, Weighted F1: {weighted_g:.4f}")
print(f"Cohere - Macro F1: {macro_c:.4f}, Weighted F1: {weighted_c:.4f}")

Gemini - Macro F1: 0.7929, Weighted F1: 0.7925
Cohere - Macro F1: 0.8032, Weighted F1: 0.8012
